In [1]:
! pip -q install python-dotenv
! pip -q install cohere
! pip -q install datasets
! pip -q install annoy

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
import cohere
co = cohere.Client(os.environ['COHERE_API_KEY'])

/home/lahad/Documents/Semantic_Wikipedia/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
from datasets import load_dataset
docs = load_dataset(f"Cohere/wikipedia-22-12-simple-embeddings", split="train")

/home/lahad/Documents/Semantic_Wikipedia/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
for item in docs:
    print(item)
    break

{'id': 0, 'title': '24-hour clock', 'text': 'The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23. It does not use a.m. or p.m. This system is also referred to (only in the US and the English speaking parts of Canada) as military time or (only in the United Kingdom and now very rarely) as continental time. In some parts of the world, it is called railway time. Also, the international standard notation of time (ISO 8601) is based on this format.', 'url': 'https://simple.wikipedia.org/wiki?curid=9985', 'wiki_id': 9985, 'views': 2450.62548828125, 'paragraph_id': 0, 'langs': 30, 'emb': [0.07711287587881088, 0.3197174072265625, -0.2051590085029602, 0.6302579045295715, 0.032093219459056854, 0.200703963637352, 0.16665680706501007, -0.31295087933540344, 0.17575109004974365, 0.5308129191398621, -0.37528499960899353, 0.3338659405708313, -0.046272162348032, 0.07841536402702332, -0.3490406274795532, 0.271

In [7]:
# Extraire les vecteurs 
embeddings = [item['emb'] for item in docs]


768

In [11]:
from annoy import AnnoyIndex

f = len(embeddings[0])
serach_index = AnnoyIndex(f,"angular")

# Ajouter des vecteurs à l'index
for i,emb in enumerate(embeddings):
    serach_index.add_item(i,emb)

# build 10 arbres
serach_index.build(10)
#save in disk
serach_index.save("index.ann")

True

In [ ]:
# fonction for semantic search

def semantic_search(query):
    query_emb = co.embed